In [1]:
import pandas as pd

<h2>Elegir el dataset de navegacion</h2>

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving navegacion.csv to navegacion.csv
User uploaded file "navegacion.csv" with length 3384875 bytes


<h2>Crear el dataframe a patir del dataset de navegacion</h2>

In [ ]:
dfNavegacion = pd.read_csv("navegacion.csv", sep=";")
print(dfNavegacion.head(10))

<h1>Analisis Exploratorio de Datos (EDA)</h1>
<h2>Limpia de datos</h2>
<p>Procedemos a eliminar todos los datos que no sean coherentes, ya que afectaria al resultado final</p>

In [ ]:
# Comprobar cuantos id_user hay repetidos
print(dfNavegacion['id_user'].value_counts().head(50))

In [ ]:
# Comprobar cuantos gclid hay repetidos
print(dfNavegacion['gclid'].value_counts().head(50))

In [ ]:
# Comprobar si los gclid corresponden a los mismos usuarios
print(dfNavegacion.groupby(['gclid','id_user'])['ts'].count())

In [ ]:
# Hemos visto que el gclid=0 pertenece a diferentes usuarios, por lo
# no es un dato coherente y lo elimino
gclid0 = dfNavegacion[dfNavegacion['gclid'] == '0'].index
print(gclid0)
dfNavegacion = dfNavegacion.drop(gclid0)

In [ ]:
# tambien vamos a eliminar el gclid que tenia 124 ocurrencias
gclid124 = dfNavegacion[dfNavegacion['gclid'] == 'EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEgJVzfD_BwE'].index
print(gclid124)
dfNavegacion = dfNavegacion.drop(gclid124)

<h1>Empezamos a procesar la URL que tengo datos que luego necesitare</h1>
<h2> separar los datos url_landing en columnas </h2>
<p> hay que asegurarse que no este vacia</p>

In [ ]:
serieUrlLanding = dfNavegacion["url_landing"][dfNavegacion['url_landing'].notnull()]
dfNavegacion[['url_base','idUser','uuid_url','camp','adg','device','sl','adv','rec','vacia']] = serieUrlLanding.str.split(pat='&',n=10,expand=True)
print(dfNavegacion.columns)

<h2>Eliminar las columnas que no voy a necesitar</h2>

In [ ]:
dfNavegacion = dfNavegacion.drop(['idUser','uuid_url','rec','vacia'], axis=1)
print(dfNavegacion.columns)

<h2>modificar la url_base para quedarnos como url/modelo </h2>

In [ ]:
#print(dfNavegacion['url_base'])
# url/modelo?parametros
dfNavegacion['url_base'] = dfNavegacion['url_base'].apply(lambda url: str(url).split('?')[0])
print(dfNavegacion['url_base'])

<h2>Obtener el modelo de coche y generar una columna con este dato</h2>

In [30]:
def obtenerModelo(url):
    posicion = url.rfind('/')
    modeloObtenido = url[posicion+1:]
    if modeloObtenido == '' or modeloObtenido == 'renting':
        modeloObtenido = 'home'
    return modeloObtenido

dfNavegacion.insert(loc=7, column='modelo', value=dfNavegacion['url_base'].apply(obtenerModelo))
print(dfNavegacion['modelo'].head(20))

<h1>Crear un orden para los usuarios</h1>
<p> Como tenemos usuarios repetidos me daria problema al hacer el merge con el dataframe de conversiones.</p>
<p> Voy a crear una columna llamada orden que almacene el orden de 
los id_user</p>
<p>Una vez tenga los datos de orden filtro el dfNavegacion para quedarme solo con el primero (orden=0)</p>

In [29]:
dfNavegacion = dfNavegacion.sort_values(['id_user','ts'])
dfNavegacion.insert(loc=1, column='orden', value=dfNavegacion.groupby('id_user')['ts'].cumcount())

# Filtrar el dataframe para quedarme solo con orden=0
dfNavegacion = dfNavegacion[dfNavegacion['orden']==0]


<h1>Unir el fichero de navegacion y el fichero de conversiones con un merge</h1>
<h2>Primero cargar el dataset de conversiones y crear dfConversiones</h2>

In [31]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving conversiones.csv to conversiones.csv
User uploaded file "conversiones.csv" with length 3165 bytes


In [ ]:
dfConversiones = pd.read_csv("conversiones.csv", sep=";")
print(dfConversiones.head(10))

<h2>Unir los dataframes a través de id_user sin perder ningun dato</h2>

In [33]:
dfFinal = dfNavegacion.merge(dfConversiones, how='outer', on=['id_user'])

<h2>Crear una nueva columa llamada conversiones: 1 (es conversion) 0 (no lo es)</h2>

In [34]:
def esConversion(dato):
    lista = ['CALL','FORM']
    if dato in lista:
        return 1
    else:
        return 0
    
dfFinal.insert(loc=1, column='conversiones', value=dfFinal['lead_type'].apply(esConversion))


<h1> *************************   PREGUNTAS A RESPONDER  ******************* </h1>

<h1>Pregunta 1: Cuantás visitas recibe en el dia el cliente?</h1>

In [36]:
#print(dfFinal.head(20))
# crear una columna dia con la fecha de la columna ts
dfFinal.insert(loc=1, column='dia', value=dfFinal['ts'].apply(lambda ts: str(ts).split(' ')[0]))
print(dfFinal.groupby('dia')['dia'].count())

dia
2021-09-06    5640
2021-09-07       4
nan             13
Name: dia, dtype: int64


<h3>Cuántas de ellas convierten y cuántas no (en %)</h3>

In [38]:
# Filtrar por 6/9/2021
visitas_dia_6_septiembre = dfFinal[dfFinal['dia'] == '2021-09-06']
num_visitas_dia = visitas_dia_6_septiembre['dia'].count()

# cuento las visitas con conversion
visitas_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones'] == 1]['ts'].count()

# cuento las visitas sin conversion
visitas_no_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones'] == 0]['ts'].count()

# calcular los porcentajes
porcentaje_visitas_convierten = round( (visitas_convierten / num_visitas_dia) * 100, 2)
porcentaje_visitas_no_convierten = round( (visitas_no_convierten / num_visitas_dia) * 100, 2)
print("Convierten:", porcentaje_visitas_convierten, '%')
print("No Convierten:", porcentaje_visitas_no_convierten, '%')

Convierten: 0.27 %
No Convierten: 99.73 %


<h1>Pregunta 2: Por tipo de conversión (CALL o FORM),  cuántas hay de cada una?</h1>

In [ ]:
print(dfFinal.groupby('lead_type')['lead_type'].count())

<h1>Pregunta 3: Porcentaje de usuarios recurrentes sobre el total de usuarios</h1>

In [40]:
# En la variable total_usuarios cuento los usuarios a través de la columna id_user
total_usuarios = dfFinal['id_user'].count()

# Filtro el dataFrame por los usuarios recurrentes con valor true y que no sean nulos. 
# Cuento los registros obtenidos a través de la columna ts y almaceno el resultado en usuarios_recurrentes
usuarios_recurrentes = dfFinal[dfFinal.user_recurrent & dfFinal.user_recurrent.notnull()]['ts'].count()

# Calculo el porcentaje y lo redondeo a 2 decimales
porcentaje = round((usuarios_recurrentes / total_usuarios) * 100, 2)
print('Usuarios recurrentes:',porcentaje, '%')


Usuarios recurrentes: 18.7 %


<h1>Pregunta 4: Coche mas visitado</h1>

In [ ]:
print(dfFinal['modelo'].value_counts())

<h2>Es el que más convierte?</h2>

In [42]:
# Voy a filtrar el dataset para tener solo los registros que han convertido y luego agruparlo
print(dfFinal[dfFinal['conversiones'] == 1].groupby('modelo')['conversiones'].count())

modelo
cea-electrico    1
clin200          2
clin400          3
dep30            1
home             5
ixs-electrico    1
tria             2
Name: conversiones, dtype: int64


<h1>Exportar el dfFinal a un Excel para elaborar informes, graficos, histogramas, ...etc</h1>

In [44]:
dfFinal.to_excel('dataframe_final.xlsx', index=False)